In [9]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Define a classification chain to categorize queries
classification_chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOllama(model="llama3.2")
    | StrOutputParser()
)

# Example usage
print(classification_chain.invoke({"question": "How do I call Anthropic?"}))  # Expected: "Anthropic"


Other


In [10]:
# LangChain sub-chain
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in LangChain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOllama(model="llama3.2")
)

# Anthropic sub-chain
anthropic_chain = (
    PromptTemplate.from_template(
        """You are an expert in Anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOllama(model="llama3.2")
)

# General question sub-chain
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOllama(model="llama3.2")
)


In [11]:
from langchain_core.runnables import RunnableLambda

# Define routing logic
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

# Create a pipeline that classifies and routes the query
full_chain = {"topic": classification_chain, "question": lambda x: x["question"]} | RunnableLambda(route)

# Example invocations
print(full_chain.invoke({"question": "How do I use Anthropic?"}))  # Routes to Anthropic chain
print("#############################")
print(full_chain.invoke({"question": "How do I use LangChain?"}))  # Routes to LangChain chain
print("#############################")
print(full_chain.invoke({"question": "What's 2 + 2?"}))  # Routes to General chain


content='Anthropic is a conversational AI tool that allows users to create interactive stories, dialogue-based games, and chatbots. Here\'s a step-by-step guide on how to use Anthropic:\n\n**Creating a New Project**\n\n1. Go to the Anthropic website ([www.anthropic.ai](http://www.anthropic.ai)) and sign up for an account.\n2. Click on the "Create" button to start a new project.\n3. Choose a template or start from scratch. Templates can help you get started quickly, while starting from scratch gives you full creative control.\n\n**Building Your Story**\n\n1. Once you\'ve chosen your template or started a new project, you\'ll be taken to the editor. This is where you build your story.\n2. Use the editor\'s drag-and-drop interface to add characters, scenes, and dialogue. You can also use pre-made assets like backgrounds and props.\n3. Write your dialogue by typing in the chat window. You can also import existing text files or paste code snippets.\n\n**Conversational Flow**\n\n1. To create

In [12]:
from langchain_core.runnables import RunnableBranch

# Define the branch-based routing logic
branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,  # Default chain
)

# Create a pipeline using `RunnableBranch`
full_chain = {"topic": classification_chain, "question": lambda x: x["question"]} | branch

# Example invocations
print(full_chain.invoke({"question": "How do I use Anthropic?"}))  # Routes to Anthropic chain
print("#############################")
print(full_chain.invoke({"question": "How do I use LangChain?"}))  # Routes to LangChain chain
print("#############################")
print(full_chain.invoke({"question": "What's 2 + 2?"}))  # Routes to General chain


content='As Harrison Chase told me, to get started with Anthropic, you first need to create an account on their website. Once you\'re logged in, you\'ll be guided through a series of steps to set up your organization and connect it to LangChain.\n\nNext, you\'ll need to install the Anthropic CLI tool, which will allow you to interact with Anthropic\'s APIs from the command line. This can be done using pip: `pip install anthropic`.\n\nAfter installation, you can use the Anthropic CLI to authenticate, retrieve your API key, and connect it to LangChain projects.\n\nTo do this, create a new file named `.anthropic.json` in your project directory and add your API key to it. Then, run the command `anthropic login --username YOUR_USERNAME --password YOUR_PASSWORD` (replace "YOUR_USERNAME" and "YOUR_PASSWORD" with your actual Anthropic credentials).\n\nThis will authenticate you with Anthropic\'s APIs, which can then be used with LangChain. You can verify the connection by running a simple scri

In [13]:
from langchain_community.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_ollama import ChatOllama

# Define subject-specific prompts
physics_template = """You are a physics professor. Explain concepts concisely.
Here is a question: {query}"""

math_template = """You are a mathematician. Break down problems into steps.
Here is a question: {query}"""

# Generate embeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Define function to choose best-matching prompt and return formatted string
def prompt_router(input):
    if isinstance(input, str):  # Ensure input is a dict
        input = {"query": input}

    query = input["query"]  # Extract query string
    query_embedding = embeddings.embed_query(query)
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    
    return most_similar.format(query=query)  # Ensure output is a string

# Create chain
chain = (
    RunnableLambda(lambda x: {"query": x})  # Convert raw input into dict
    | RunnableLambda(prompt_router)  # Routes to correct prompt
    | ChatOllama(model="llama3.2")  # Processes query with Ollama
    | StrOutputParser()  # Extracts output as a string
)

# Example usage
print(chain.invoke("What’s a black hole?"))  # Routes to Physics chain
print("#############################")
print(chain.invoke("What’s a path integral?"))  # Routes to Math chain


A black hole is a region in space where the gravitational pull is so strong that nothing, including light, can escape. It's formed when a massive star collapses in on itself and its gravity becomes so intense that it warps the fabric of spacetime.

Think of it like this: imagine spacetime as a trampoline. If you place a heavy object on it, it will warp and curve. A black hole is like a cosmic sinkhole where the gravitational pull is so strong that it creates an impenetrable boundary called the event horizon. Once something crosses the event horizon, it's trapped forever.

Black holes have three main properties: mass, charge, and angular momentum. The mass of a black hole determines its strength of gravity, while its charge and angular momentum determine how it behaves in the presence of other matter and radiation.

Now, that's a concise introduction to black holes!
#############################
A great question in the realm of quantum mechanics and mathematics!

As a mathematician, I'd